## Notebook to combine all feature engineered data

In [1]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
from functools import reduce

In [2]:
for i in range (2003, 2020):
    
    year=str(i)
    
    PATH_1=os.path.join('..','all_years','CAMS', year ,'feature_engineering')

    # list all the pkl files in the folder
    pkl_files = [file for file in os.listdir(PATH_1) if file.endswith('.pkl')]

    ## read all the pickle files
    dataframes = [pd.read_pickle(os.path.join(PATH_1, file)) for file in pkl_files]

    ## remove the NAME columns from the dataframes. This step was necessary because, functools (below) didnt work
    ## when dataframes had same columns

    for j in range(len(dataframes)):
        if "NAME" in dataframes[j].columns:
            dataframes[j] = dataframes[j].drop(columns="NAME")
   
    #from functools import reduce. merge all the datframes in the list on fips
    df1=reduce(lambda x, y: pd.merge(x, y, on = 'fips'), dataframes)
    df1

    ## get the pm25 above threshold data which is on a different folder
    PATH_2=os.path.join('..','all_years','CAMS', year, 'FE_PM25_'+ year +'.pkl')
    df2=pd.read_pickle(PATH_2)
    df2 = df2.drop('NAME', axis=1)
    df2

    final_df=df1.merge(df2,how='left', on='fips')
    final_df['year']=int(i)
    final_df
    
    ## save in the feature_engineered file
    PATH_3='feature_engineered'
    file=os.path.join(PATH_3, 'fe_'+ year +'.pkl')
    final_df.to_pickle(file)
